In [58]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from matplotlib import pyplot as plt
import math 

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2023-08-15 15:48:22.395008: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 15:48:22.412225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 15:48:22.412378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

2023-08-15 15:48:23.243060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 15:48:23.243241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 15:48:23.243303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [83]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    #iki ayak arasındaki mesafeyi yazdırma
    font = cv2.FONT_HERSHEY_SIMPLEX
    y3,x3,c3 =shaped[15]
    y4,x4,c4 =shaped[16]
    uzunluk = round(math.sqrt(pow(x3-x4,2)+pow(y3-y4,2)),4)
    uzunluk = str(uzunluk)
    if (c3 > confidence_threshold) & (c4 > confidence_threshold):      
            cv2.line(frame, (int(x3), int(y3)), (int(x4), int(y4)), (255,255,0), 1)
            cv2.putText(frame,uzunluk ,(int(x3)+10, int(y3)+10),font,1,(255,0,0),2,cv2.LINE_AA )
            
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [84]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)


In [85]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [88]:
cap = cv2.VideoCapture('/home/muhlabws4/Belgeler/tkd/tkd.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [33]:
results

{'output_0': <tf.Tensor: shape=(1, 6, 56), dtype=float32, numpy=
 array([[[0.43991423, 0.06092735, 0.4457217 , 0.42901307, 0.06069944,
          0.6499668 , 0.4285391 , 0.06025588, 0.6309547 , 0.440482  ,
          0.06126946, 0.61149234, 0.4366787 , 0.06406694, 0.49852806,
          0.48452136, 0.06468624, 0.75515556, 0.47395143, 0.09692328,
          0.595781  , 0.5706534 , 0.08224839, 0.6438702 , 0.55889887,
          0.09427242, 0.3753593 , 0.574195  , 0.06051747, 0.2588045 ,
          0.5515755 , 0.0653329 , 0.33367053, 0.6126305 , 0.10748235,
          0.8533598 , 0.60436535, 0.13439038, 0.72087884, 0.704488  ,
          0.06538225, 0.7165741 , 0.70672816, 0.1597046 , 0.7921108 ,
          0.79825085, 0.03323438, 0.6792437 , 0.7917177 , 0.19790028,
          0.69137055, 0.39854828, 0.00307271, 0.83937484, 0.21327272,
          0.5875611 ],
         [0.4554674 , 0.40234432, 0.47407597, 0.44714278, 0.40723696,
          0.49584794, 0.44787127, 0.4020376 , 0.58597666, 0.44968224,
  

In [28]:
keypoints_with_scores


array([[[0.41088817, 0.24954994, 0.4523373 ],
        [0.3945577 , 0.24438123, 0.45210546],
        [0.39915   , 0.25101152, 0.5338396 ],
        [0.40267396, 0.23249061, 0.5800451 ],
        [0.4061062 , 0.24648848, 0.5713766 ],
        [0.44868928, 0.20465843, 0.6985268 ],
        [0.45751464, 0.25324604, 0.814281  ],
        [0.53422046, 0.19113109, 0.6325642 ],
        [0.5267885 , 0.265304  , 0.52021015],
        [0.5657733 , 0.19673787, 0.22904326],
        [0.566689  , 0.29985562, 0.58560973],
        [0.58400214, 0.20509723, 0.8072288 ],
        [0.5798999 , 0.24040714, 0.905047  ],
        [0.69274294, 0.18410268, 0.86238265],
        [0.6895544 , 0.27681416, 0.48151615],
        [0.7886228 , 0.15727326, 0.84906006],
        [0.79095477, 0.3043884 , 0.7742408 ]],

       [[0.4119987 , 0.44233784, 0.4691839 ],
        [0.40489012, 0.44694445, 0.3456158 ],
        [0.40476686, 0.44226918, 0.43437576],
        [0.41128752, 0.46060595, 0.41418093],
        [0.40434775, 0.44568214,